In [ ]:
# TODO: test new model.evaluate, DataMap
import numpy as np

from nems import Model
from nems.layers import Layer

# None, 'str', and ['list'] should show up in DataMap.args
# {'dict'} should show up in DataMap.kwargs
# Both should statements should still work for nested structures, and ordering
# shouldn't matter.
input_tests = [
    None,
    'str',
    ['list1'],
    [None, 'list2'],
    ['list3', None],
    ['list4', ['list5', 'list5']],
    {'dict1': 'x', 'dict2': 'y'},
    {'dict3': None},
    ['list6', {'dict4': 'z'}]
    ]

output_tests = [
    None,
    'str',
    ['list1'],
    ['list2', 'list3'],
]

class PassThrough(Layer):
    def evaluate(self, input):
        out = np.array([[f'from {self.name}']])
        if isinstance(input, (list)):
            out = [out]*len(input)
        return out

class Duplicate(Layer):
    def evaluate(self, input):
        return [np.array([[f'from {self.name}']])]*2

class JustOne(Layer):
    def evaluate(self, input_list):
        assert isinstance(input_list, list)
        return np.array([[f'from {self.name}']])


In [ ]:
for input in input_tests:
    print(f'input is: {input}')
    print(PassThrough(input=input).get_input_map())

In [ ]:
# Last one should raise an error since there's only one output
# but Layer.output has 2 entries
for output in output_tests:
    print(f'output is: {output}')
    layer = PassThrough(output=output)
    _map = layer.get_input_map()
    _map.map_outputs('output')
    print(_map)

In [ ]:
input = {'input': np.array([['input']])}

model = Model()
model.add_layers(
    # input should be 'input', the Model default
    PassThrough(name='p1', input=None, output='str'), 
    PassThrough(name='p2', input='str', output=['list1']),
    PassThrough(name='p3', input=None, output=None),
    Duplicate(name='d1', input=None, output=['list2', 'list3']),
    # should map to ['str', 'str.1']. need double [[]] b/c we want a list as a
    # single argument, not two list elements as two arguments
    PassThrough(name='p4', input=[['list2', 'list3']], output='str'),
    # should be same as input=None in this case, and output should map to
    # 'output' (the Model default). In this case, only need a single [] b/c
    # the list is already mapped to a single argument (named 'input_list').
    JustOne(name='i1', input={'input_list': ['str', 'str.1']}, output=None)  
)

eval_data = model.evaluate(input)
dm = eval_data.pop('_data_map')
dm

In [ ]:
model.print_layer_data(input)

In [ ]:
import numpy as np
from nems import Model
# try with random data
input  = {'stimulus': np.random.rand(100, 18),
          'pupil': np.random.rand(100, 1)}
eval_kwargs = {'input_name': 'stimulus', 'state_name': 'pupil'}
model2 = Model.from_keywords('wc.18x4.g-stategain.1x4-fir.4x15-dexp')
model2.print_layer_data(input, show_full_data=False, **eval_kwargs)

In [ ]:
# TODO: add docs for new evaluate stuff
# TODO: test updated changes